# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'Snickers'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
dadosteste = pd.read_excel('Snickers (teste).xlsx')
dadostreino = pd.read_excel('Snickers (treino).xlsx')
inutil=dadostreino.Treinamento[dadostreino.prob==3]
util=dadostreino.Treinamento[dadostreino.prob<3]

tudo=dadostreino.Treinamento

testeinutil=dadosteste.Teste[dadosteste.prob==3]
testeutil=dadosteste.Teste[dadosteste.prob<3]
tudoteste=dadosteste.Teste

In [8]:
def crialista(x):
    listatreino=[]

    for i in x:
        listatreino.append(i.split())
    return listatreino

In [9]:
def formatlist(x):
    newWord=0

    remove = ['/','?','.',',',';',':','\\',',','+','@','#',"'","\"","!","(",")"]

    removeInWordFull = ['http',"rt"]    #ira remover a palavra inteira

    removeInWordSlice = ['/','?','.',',',';',':','\\',',','+','@','#',"'","\"","!","(",")"]     #ira remover o caractere da mensagem

    inut = 0

    for i in range(0,len(x)): #procurar cada mensagem [i index da mensagem]
        for j in range(0,len(x[i])): #procurara cada palavra na mensagem [j index da palavra]

            if x[i][j] in remove:         #remove a palavra inteira se bater com alguma do "remove"
                inut+=1
                x[i][j]="remuvido"

            for l in range(0,len(x[i][j])): #procurara cada letra na palavra [l index da letra]

                if any(word in x[i][j] for word in removeInWordFull):
                    inut+=1
                    x[i][j]="remuvido"          #remove se tiver sequencia de caracteres indesejados 
                    break

                '''
                if listatreino[i][j][l] in removeInWordFull:
                    listatreino[i][j]="remuvido"                  #remove se tiver caractere indesejado
                    break
                '''

            for l in range(0,len(x[i][j])): #procurara cada letra na palavra [l index da letra]
                if x[i][j][l] in removeInWordSlice:
                    newWord = list(x[i][j])
                    inut+=1
                    del newWord[l]
            if (newWord):
                x[i][j]=''.join(newWord)
            newWord=0


    for i in range(0,len(x)): #procurar cada mensagem [i index da mensagem] 
        deleted=0
        for j in range(0,len(x[i])): #procurara cada palavra na mensagem [j index da palavra]
            if x[i][j+deleted]=="remuvido": #deleta todos os "remuvido"
                del x[i][j+deleted]
                deleted-=1
    #print ("foram removidos {} caracteres".format(inut))  
    return x

In [10]:
def prob(x) :
    probDict = []

    for i in range(0,len(x)): #procurar cada mensagem [i index da mensagem]
        for j in range(0,len(x[i])): #procurara cada palavra na mensagem [j index da palavra]
            ach=0
            for k in range(0,len(probDict)):
                if probDict[k][0] == x[i][j]:
                    probDict[k][1]+=1
                    ach=1

            if ach==0:
                probDict.append([x[i][j],1])
            ach=0
    return probDict

In [11]:
def formatprob(x):
    NumTotal = 0
    for i in x:
        NumTotal += i[1]
    for i in range(len(x)):
        x[i] = [x[i][0],x[i][1],x[i][1]/NumTotal]
    probDict_palavras = []
    for i in range(len(x)):
        probDict_palavras.append(x[i][0])
    return ([x,probDict_palavras])

In [12]:
def multi(x):
    resp=1
    for i in x:
        resp*=i
    return resp

In [13]:
def probfrase(frase,probDict_palavras,probDict):
    resp=[]
    for i in frase:
        prob_frase = []
        for j in i:
            if j in probDict_palavras:
                ind = probDict_palavras.index(j)
                prob_frase.append(probDict[ind][2])
            else:
                prob_frase.append(1)
        resp.append([frase[frase.index(i)],multi(prob_frase)])
    return resp

In [14]:
def bott(j,study=0):
    resp=[]
    for x in j:
        lista=crialista(x)
        lista=formatlist(lista)
        lista=formatlist(lista)
        problist=prob(lista)
        problist=formatprob(problist)[0]
        probpalavras=formatprob(problist)[1]
        resp.append([lista,problist,probpalavras])
    return resp

In [15]:
def ageitapa(x):
    lista=crialista(x)
    lista=formatlist(lista)
    lista=formatlist(lista)
    return lista

In [16]:
def botr(fraseb,x):
    frasey=ageitapa(fraseb)
    resp=[]
    for i in x:
        resp.append(probfrase(frasey,i[2],i[1]))
    return resp

In [17]:
def formatbotr(x):
    
    resputil=[]
    respinutil=[]
    for i in range(0,len(x[1])):
        if x[0][i][1] > x[1][i][1]:
            respinutil.append(x[0][i][0])
        else:
            resputil.append(x[0][i][0])
    return(resputil,respinutil)

In [20]:
#formatbotr (botr ( lista de palavras no pandas , treino do bot)) [0 = util, 1 = inutil]
print("  TREINO")
print("pessoa")
print((inutil), " inutil")
print((util), " uteis")

print("bot")
print((formatbotr(botr(tudo,bott([util,inutil],1)))[1]), " inuteis") 
print((formatbotr(botr(tudo,bott([util,inutil],1)))[0]), " uteis")



print("\n"," TESTE")
print("pessoa")
print((testeinutil), " inutil")
print((testeutil), " uteis")

print("bot")
print((formatbotr(botr(tudoteste,bott([util,inutil],1)))[1]), " inuteis") 
print((formatbotr(botr(tudoteste,bott([util,inutil],1)))[0]), " uteis")

  TREINO
pessoa
0               @e3truelovers ah o twix e os snickers tb
11     vcs viram que eu virei garota propaganda da sn...
13     rt @porracavalcante: a pessoa mais sexy comend...
17     saudades da época que com r&amp;5,00 a gente c...
27                               ganhei um snickers hoje
29     se postar foto de @snickers no stories do inst...
30         rt @chsnectar: queria so um snickers agora...
31     @evertonferrer_ @gabrielsilsan_ ok mas cadê o ...
34     rt @jackposobiec: ana navarro https://t.co/c41...
37     serio\nnem meu snickers de todo o santo dia eu...
39     tu e ele amam-se porque não estam juntos de no...
42     eu fiquei muito sem graça quando ela apareceu ...
44     muitos parabéns :d — obrigadooo https://t.co/g...
56     comendo snickers pela primeira vez https://t.c...
63     vocês eram o meu casal #goals 😍😍 — obrigado 😅😅...
64     @rvfvquintana tu só quer que eu coma snickers,...
78     atribuí nota 8 ao episódio 5x7 - full bush, ha...
80     esse sni

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

Porcentagem de positivos falsos = 31
Porcentagem de positivos verdadeiros = 129
Porcentagem de negativos verdadeiros = 18
Porcentagem de negativos falsos = 22

acerto em analises = 73.5%
erros em analises = 26.5%

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


<p>   Concuimos que nosso bot conseguiu chegar razoavelmente perto dos resultados que nos obtivemos quando nos mesmos analisamos. </p> 
<p>   A sua porcentagem de acerto foi bem alta. Considerando que a analise humana pode ter ocorrido algumas falhas, nos esperavamos uma porcentagem de acerto um pouco menor por ser um programa de analise tão primitivo em relacão as suas condicões analiticas.</p> 
<p>O bot analisou 200 frases que ja haviam sido analisadas por nos, alunos, definindo elas como relevantes ou não. Tomando como a analise humana 100% certa, o bot acertou 73.5% das analises (147 frases), e errou 26.5% das analises (53 frases).</p> 
<p>Como nosso programa de analise ainda é muito rudimentar, sem parâmentros de análise complexos, não poderiamos esperar uma porcentagem de acerto tão elevado.</p> 
<p>Para cada frase nos focamos em analisar palavra por palavra, assim coisas como sarcasmo podem acabar sendo confundidas e mal-interpretadas pelo programa.</p> 
<p>Uma possível expansão para o programa seria justamento a interpretação avançada do sarcasmo, e também expanidir para identificação de posição em relação ao produto, onde o programa falaria também, alem de ser util ou não, mas de dizer se é um util com características positivas, ou negativas.</p> 
<p>Esse programa pode ser usado para empresas de médio e grande porte para avaliar seu produto com o cliente, e o que as pessoas pensam dele. Podendo expandir para outras redes sociais se houver necessidade.</p> 